In [0]:
%%bash
pip install --upgrade tensorflow-gpu

  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.


In [0]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

tf.__version__

'2.0.0'

In [0]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
##Copy data to local
%%bash
cp '/content/drive/My Drive/Colab Notebooks/TF Keras 2.0 GRIR - CMLE + FeatEngg/GRIR_GCP_Data.csv' 'GRIR_GCP_Data.csv'

In [0]:
#Read data set CSV
df = pd.read_csv("GRIR_GCP_Data.csv", sep=",")
df.head(5)

,WERKS,SCENARIO,KTOKK,VSTATU,VPATD,EKORG,EKGRP,TOTGRQTY,TOTIRQTY,NODLGR,NODLIR,DIFGRIRD,DIFGRIRV,STATUS
0,ML01,3,1,1,30,1,A,0,80,0,90,-80,-38100,1
1,ML01,3,1,1,30,1,A,0,107,0,177,-107,-41600,0
2,ML01,3,1,1,30,1,A,0,107,0,152,-107,-27600,1
3,ML01,3,1,1,30,1,A,0,96,0,79,-96,-13800,1
4,ML01,3,1,1,30,1,A,0,146,0,192,-146,-73500,0


In [0]:
#Mark some columns as categorical
for col_cat in ['SCENARIO','KTOKK','VSTATU','EKORG']:
    df[col_cat] = df[col_cat].astype('category')
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8279 entries, 0 to 8278
Data columns (total 14 columns):
WERKS       8279 non-null object
SCENARIO    8279 non-null category
KTOKK       8279 non-null category
VSTATU      8279 non-null category
VPATD       8279 non-null int64
EKORG       8279 non-null category
EKGRP       8279 non-null object
TOTGRQTY    8279 non-null int64
TOTIRQTY    8279 non-null int64
NODLGR      8279 non-null int64
NODLIR      8279 non-null int64
DIFGRIRD    8279 non-null int64
DIFGRIRV    8279 non-null int64
STATUS      8279 non-null int64
dtypes: category(4), int64(8), object(2)
memory usage: 679.7+ KB


In [0]:
#Prepare train and output columns
df_x = df.drop(['STATUS'],axis=1)
df_y = df['STATUS']

In [0]:
#Split Train and Validation

#Use the best split value from above after manual inspection
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.1, random_state=8,stratify=df_y)
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7451 entries, 692 to 315
Data columns (total 13 columns):
WERKS       7451 non-null object
SCENARIO    7451 non-null category
KTOKK       7451 non-null category
VSTATU      7451 non-null category
VPATD       7451 non-null int64
EKORG       7451 non-null category
EKGRP       7451 non-null object
TOTGRQTY    7451 non-null int64
TOTIRQTY    7451 non-null int64
NODLGR      7451 non-null int64
NODLIR      7451 non-null int64
DIFGRIRD    7451 non-null int64
DIFGRIRV    7451 non-null int64
dtypes: category(4), int64(7), object(2)
memory usage: 611.7+ KB


In [0]:
#Save train and test CSV
df_train = pd.concat([x_train, y_train], axis = 1)
df_train.to_csv("train.csv", index =False)

df_test = pd.concat([x_test, y_test], axis = 1)
df_test.to_csv("test.csv", index=False)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7451 entries, 692 to 315
Data columns (total 14 columns):
WERKS       7451 non-null object
SCENARIO    7451 non-null category
KTOKK       7451 non-null category
VSTATU      7451 non-null category
VPATD       7451 non-null int64
EKORG       7451 non-null category
EKGRP       7451 non-null object
TOTGRQTY    7451 non-null int64
TOTIRQTY    7451 non-null int64
NODLGR      7451 non-null int64
NODLIR      7451 non-null int64
DIFGRIRD    7451 non-null int64
DIFGRIRV    7451 non-null int64
STATUS      7451 non-null int64
dtypes: category(4), int64(8), object(2)
memory usage: 669.9+ KB


In [0]:
def feature_engg_features(features):
  #Add new features
  features['grminusirbyvpatd'] = ( features['TOTGRQTY'] - features['TOTIRQTY'] ) / features['VPATD']

  return(features)

In [0]:
def feature_engg(features, label):
  #Add new features
  features = feature_engg_features(features)

  return(features, label)

In [0]:
# Determine CSV, label, and key columns
#Columns in training sheet -> Can have extra columns too
CSV_COLUMNS = df_train.columns
LABEL_COLUMN = 'STATUS'

# Set default values for each CSV column( Including Y column )
DEFAULTS = [["ML01"], ['0'], ['0'],["0"],[-1],["-1"],["-1"],[0],[0],[0],[0],[0],[0],[0]]

def make_input_fn(filename, mode, vnum_epochs = None, batch_size = 512):
    def _input_fn(v_test=False):     
        # Create list of files that match pattern
        file_list = tf.io.gfile.glob(filename)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = vnum_epochs # indefinitely
        else:
            num_epochs = 1 # end-of-input after this        
        
        # Create dataset from file list
        dataset = tf.compat.v1.data.experimental.make_csv_dataset(file_list,
                                                   batch_size=batch_size,
                                                   column_names=CSV_COLUMNS,
                                                   column_defaults=DEFAULTS,
                                                   label_name=LABEL_COLUMN,
                                                   num_epochs = num_epochs,
                                                   num_parallel_reads=30)
        
        #Feature engineering
        dataset = dataset.map(feature_engg)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = vnum_epochs # indefinitely
            dataset = dataset.shuffle(buffer_size = batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs)
        dataset = dataset.prefetch(buffer_size = batch_size)
        
        #Begins - Uncomment for testing only -----------------------------------------------------<
        if v_test == True:
          x, y = dataset.make_one_shot_iterator().get_next()
          #TF 2.0 has eager execution
          print(x, y)
          
        #End - Uncomment for testing only -----------------------------------------------------<
        return dataset
    return _input_fn

In [0]:
#Test dataset read + Feat Engg function's - output's CSV + Feature engg columns
eval_file = "test*.csv"
fn_d = make_input_fn(filename = eval_file,
                    mode = tf.estimator.ModeKeys.TRAIN,
                    vnum_epochs = 1,
                    batch_size = 10)

fn_d(v_test=True)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
OrderedDict([('WERKS', <tf.Tensor: id=146, shape=(10,), dtype=string, numpy=
array([b'ML02', b'ML01', b'ML01', b'ML03', b'ML01', b'ML03', b'ML02',
       b'ML01', b'ML02', b'ML02'], dtype=object)>), ('SCENARIO', <tf.Tensor: id=141, shape=(10,), dtype=string, numpy=
array([b'4', b'4', b'4', b'1', b'3', b'3', b'2', b'4', b'3', b'3'],
      dtype=object)>), ('KTOKK', <tf.Tensor: id=138, shape=(10,), dtype=string, numpy=
array([b'1', b'2', b'1', b'1', b'1', b'2', b'2', b'2', b'1', b'2'],
      dtype=object)>),

<DatasetV1Adapter shapes: (OrderedDict([(WERKS, (None,)), (SCENARIO, (None,)), (KTOKK, (None,)), (VSTATU, (None,)), (VPATD, (None,)), (EKORG, (None,)), (EKGRP, (None,)), (TOTGRQTY, (None,)), (TOTIRQTY, (None,)), (NODLGR, (None,)), (NODLIR, (None,)), (DIFGRIRD, (None,)), (DIFGRIRV, (None,)), (grminusirbyvpatd, (None,))]), (None,)), types: (OrderedDict([(WERKS, tf.string), (SCENARIO, tf.string), (KTOKK, tf.string), (VSTATU, tf.string), (VPATD, tf.int32), (EKORG, tf.string), (EKGRP, tf.string), (TOTGRQTY, tf.int32), (TOTIRQTY, tf.int32), (NODLGR, tf.int32), (NODLIR, tf.int32), (DIFGRIRD, tf.int32), (DIFGRIRV, tf.int32), (grminusirbyvpatd, tf.float64)]), tf.int32)>

In [0]:
# Define feature columns
def create_feature_cols():
#   lat_buck = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('latitude'), 
#                                                  boundaries = np.arange(32.0, 42, 1).tolist())
#   long_buck = tf.feature_column.bucketized_column(tf.feature_column.numeric_column('longitude'),
#                                                   boundaries = np.arange(1, 52, 1).tolist())
    werks_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='WERKS',
            vocabulary_list=['ML01','ML02','ML03'])
    scenario_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='SCENARIO',
            vocabulary_list=['1','2','3','4'])
    ktokk_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='KTOKK',
            vocabulary_list=['1','2'])    
    vstatu_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='VSTATU',
            vocabulary_list=['1','2'])
    ekorg_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='EKORG',
            vocabulary_list=['1','2'])   
    ekgrp_c = tf.feature_column.categorical_column_with_vocabulary_list(
            key='EKGRP',
            vocabulary_list=['A','B','C'])

    return [
        tf.feature_column.indicator_column(werks_c),
        tf.feature_column.indicator_column(scenario_c),
        tf.feature_column.indicator_column(ktokk_c),
        tf.feature_column.indicator_column(vstatu_c),
        tf.feature_column.indicator_column(ekorg_c),
        tf.feature_column.indicator_column(ekgrp_c),
        tf.feature_column.numeric_column('VPATD'),
        tf.feature_column.numeric_column("TOTGRQTY"),
        tf.feature_column.numeric_column("TOTIRQTY"),
        tf.feature_column.numeric_column("NODLGR"),
        tf.feature_column.numeric_column("NODLIR"),
        tf.feature_column.numeric_column("DIFGRIRD"),
        tf.feature_column.numeric_column("DIFGRIRV"),
        tf.feature_column.numeric_column("grminusirbyvpatd")
  ]

create_feature_cols()

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='WERKS', vocabulary_list=('ML01', 'ML02', 'ML03'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='SCENARIO', vocabulary_list=('1', '2', '3', '4'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='KTOKK', vocabulary_list=('1', '2'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='VSTATU', vocabulary_list=('1', '2'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='EKORG', vocabulary_list=('1', '2'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='EKGRP', vocabulary_list=('A', 'B', 'C'), dtype=tf.string, default_value=-1

# **Keras: Create dataset + model**

In [0]:
def create_keras_model(params, feature_cols):
    model = tf.keras.Sequential()

    #First step is feature engineering
    # model.add(tf.keras.backend.map_fn(feature_engg_features))

    #Tensorflow style feature columns
    model.add(tf.keras.layers.DenseFeatures(feature_cols))

    #Other columns
    model.add(tf.keras.layers.Dense(32, activation='relu',kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.BatchNormalization())

    for l_ in range(params['hidden_layers']):
        model.add(tf.keras.layers.Dense(32, activation='relu',kernel_initializer='he_uniform'))
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(1,   activation='sigmoid'))
    return model

In [0]:
def keras_train_and_evaluate(model, params, train_dataset, validation_dataset):
  #Set optimizer
  opt = tf.keras.optimizers.Adam(lr= params['lr'], beta_1=params['beta_1'], 
                                       beta_2=params['beta_2'], epsilon=params['epsilon'])

  #Compile model
  model.compile(loss='binary_crossentropy',  optimizer=opt, metrics =['accuracy'])

  #Print Summary
  # print(model.summary())

  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001, verbose = 1)

  #Train
  out = model.fit_generator(train_dataset, 
                  validation_data = validation_dataset, 
                  epochs=100,
                  validation_steps = 2,   ###Has to be passed - Cant help it :)
                  steps_per_epoch = 10,   ###Has to be passed - Cant help it :) [ Number of batches per epoch ]
                  callbacks=[reduce_lr, keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)])

In [0]:
# epochs = 100
#Create dataset input functions
train_dataset = make_input_fn(filename = 'train*.csv',
                    mode = tf.estimator.ModeKeys.TRAIN,
                    # vnum_epochs = epochs,
                    batch_size = 512)()

validation_dataset = make_input_fn(filename = 'test*.csv',
                    mode = tf.estimator.ModeKeys.EVAL,
                    # vnum_epochs = 1,
                    batch_size = 512)()

#Create model
params_default = {
    'lr' : 0.01,
    'beta_1' : 0.99,
    'beta_2' : 0.999,
    'epsilon' : 1e-08,
    'decay' : 0.01,
    'hidden_layers' : 1
}

m_ = create_keras_model(params = params_default, feature_cols = create_feature_cols())

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [0]:
keras_train_and_evaluate(m_, params_default, train_dataset, validation_dataset)

Epoch 1/100
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
10/10 [==============================] - 8s 847ms/step - loss: 0.6127 - accuracy: 0.6035 - val_loss: 1.3005 - val_accuracy: 0.6087
Epoch 2/100
10/10 [==============================] - 2s 153ms/step - loss: 0.5159 - accuracy: 0.7252 - val_loss: 1.0906 - val_accuracy: 0.6087
Epoch 3/100
10/10 [==============================] - 1s 142ms/step - loss: 0.4770 - accuracy: 0.7598 - val_loss: 0.9594 - val_accuracy: 0.6087
Epoch 4/100
10/10 [==============================] - 1s 141ms/step - loss: 0.4499 - accuracy: 0.7694 - val_loss: 0.7988 - val_accuracy: 0.6087
Epoch 5/100
10/10 [==============================] - 1s 109ms/step - loss: 0.4158 - accuracy: 0.7801 - val_loss: 0.6821 - val_accuracy: 0.6014
Epoch 6/100
10/10 [==================

## **Convert to Estimator**

In [0]:
# Serving function for external call
def serving_fn():
    feature_placeholders  = {'WERKS' : tf.placeholder(tf.string, [None]),
            'SCENARIO' : tf.placeholder(tf.string, [None]),
            'KTOKK' : tf.placeholder(tf.string, [None]),
            'VSTATU' : tf.placeholder(tf.string, [None]),
            'EKORG' : tf.placeholder(tf.string, [None]),
            'EKGRP' : tf.placeholder(tf.string, [None]),
            'VPATD' : tf.placeholder(tf.float32, [None]),
            'TOTGRQTY' : tf.placeholder(tf.float32, [None]),
            'TOTIRQTY' : tf.placeholder(tf.float32, [None]),
            'NODLGR' : tf.placeholder(tf.float32, [None]),
            'NODLIR' : tf.placeholder(tf.float32, [None]),
            'DIFGRIRD' : tf.placeholder(tf.float32, [None]),
            'DIFGRIRV' : tf.placeholder(tf.float32, [None])
    }

    #Features with transformation logic
    features = {
                key: tf.expand_dims(tensor, -1)
                for key, tensor in feature_placeholders.items()
            }
    
    #feat_changed = add_engineered(features.copy())
    return tf.estimator.export.ServingInputReceiver(feature_engg_features(features), feature_placeholders )

In [0]:
# Create estimator train and evaluate function
def est_train_and_evaluate(keras_model, output_dir, num_train_steps, train_file, eval_file):    
    run_config = tf.estimator.RunConfig(save_checkpoints_secs = 40, 
                                        keep_checkpoint_max = 10)
    
    #Convert Keras model to TF estimator
    keras_model.compile()
    estimator = tf.keras.estimator.model_to_estimator(keras_model=keras_model, config=run_config)

    train_spec = tf.estimator.TrainSpec(input_fn =  make_input_fn(filename = train_file,
                                                    mode = tf.estimator.ModeKeys.TRAIN,
                                                    batch_size = 512),
                                        max_steps = num_train_steps)
    
    #Create exporter
    # exp = tf.estimator.LatestExporter("decision", serving_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn =    make_input_fn(filename = eval_file,
                                                    mode = tf.estimator.ModeKeys.TRAIN,
                                                    batch_size = 512),
                                      steps = None, 
                                      # exporters = exp,
                                      start_delay_secs = 20, # start evaluating after N seconds, 
                                      throttle_secs = 45)  # evaluate every N seconds

                                
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [0]:
#tf.logging.set_verbosity(tf.logging.WARN)
train_file = "train*.csv"
eval_file = "test*.csv"

est_train_and_evaluate(m_, None, 20000, train_file, eval_file)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9ratcs23', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 40, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f342fa6cc88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinat

KeyError: ignored